# Chapter 2

In this chapter we will start witht the basic concepts of Object Oriented Programming for the Options.

## Implementing the pay-off class

We create a class that depending on the Option type, the Strike price and the "Spot" (stock price in the future), calculates the pay off.

In [38]:
class PayOff():
    def __init__(self,TheOptionsType_,Strike_):
        # The constructor
        self.__TheOptionsType = TheOptionsType_
        self.__Strike = Strike_
        
    def __call__(self,spot):
        # Overloading the ( ) operator
        if self.__TheOptionsType == 'call':
            return max(spot - self.__Strike,0)
        elif self.__TheOptionsType == 'put':
            return max(self.__Strike - spot,0)
        else:
            raise Exception('Unknown option type found.')
        

Time to test it!

In [21]:
call = PayOff('call',80)
callPrice  = call(100)
print(f'Call price: {callPrice}')

Call price: 20


In [22]:
put = PayOff('put',80)
putPrice  = put(100)
print(f'Put price: {putPrice}')

Put price: 0


In [23]:
err = PayOff('err',80)
errPrice  = err(100)
print(f'Err price: {errPrice}')

Exception: Unknown option type found.

The class is behaving exactly as we wanted.

## Using the pay-off class

Now we will play with that class inside the code of the chapter 1 with the monte carlo routine

In [34]:
from random import random
from math import exp, sqrt, log

In [50]:
def SimpleMonteCarlo2(thePayOff,
                     Expiry,
                     Spot,
                     Vol,
                     r,
                     NumberOfPaths):
    
    variance = Vol*Vol*Expiry
    rootVariance = sqrt(variance)
    itoCorrection = -0.5*variance
    
    movedSpot = Spot*exp(r*Expiry + itoCorrection)
    runningSum = 0
    for i in range(NumberOfPaths):
        thisGaussian = GetOneGaussianByBoxMuller()
        thisSpot = movedSpot*exp(rootVariance*thisGaussian)
        thisPayoff = thePayOff(thisSpot)
        runningSum += thisPayoff
    
    mean = runningSum / NumberOfPaths
    mean *= exp(-r*Expiry)
    return mean      

In [27]:
def GetOneGaussianBySummation():
    result = 0
    for j in range(12):
        result += random()
    result -= 6
    return result

def GetOneGaussianByBoxMuller():
    sizeSquared = 1
    while sizeSquared >= 1:
        x = 2*random() - 1
        y = 2*random() - 1
        sizeSquared = x*x + y*y
    
    result = x*sqrt(-2*log(sizeSquared)/sizeSquared)
    return result

In [51]:
callPayOff = PayOff('call',40)
callPrice = SimpleMonteCarlo2(callPayOff,0.5,42,0.2,0.1,10000)
print(f'Call price: {round(callPrice,2)}, for 10000 paths')

Call price: 4.82, for 10000 paths


Which it was what we were getting before in Chapter 1.

### Extensibility problems

Now if we want to add new features we do not have to touch our Monte Carlo pricer, however we still have to touch part of the code in the class, in the next chapter we will use the concept of inheretance to avoid doing so.

### Exercises

**Exercise 2.1** Modify the pay-off class so that it can handle digital options

Digial options return a 0 or a 1 instead of the difference between the strike price and the price. 

So as example, let's take a call option. In maturity if the underlying asset price is 40 and the strike 38 the price of the option will be 2, if the strike price is 42, the price of the option will be 0.

In the case of a digital option, in the first case it would return 0, whereas in the second it will return 0.

In [75]:
class PayOff():
    def __init__(self,TheOptionsType_,Strike_):
        # The constructor
        self.__TheOptionsType = TheOptionsType_
        self.__Strike = Strike_
        
    def __call__(self,spot):
        # Overloading the ( ) operator
        if self.__TheOptionsType == 'call':
            return max(spot - self.__Strike,0)
        elif self.__TheOptionsType == 'put':
            return max(self.__Strike - spot,0)
        elif self.__TheOptionsType == 'digital call':
            if spot >= self.__Strike:
                return 1
            else:
                return 0
        elif self.__TheOptionsType == 'digital put':
            if spot <= self.__Strike:
                return 1
            else:
                return 0
        else:
            raise Exception('Unknown option type found.')

We will use the example in the following page to test our results: https://www.quantstart.com/articles/Digital-option-pricing-with-C-via-Monte-Carlo-methods

With the following data and results:
* Number of Paths: 10000000
* Underlying:      100
* Strike:          100
* Risk-Free Rate:  0.05
* Volatility:      0.2
* Maturity:        1
* Call Price:      0.532424
* Put Price:       0.418726 
    

In [76]:
callPayOff = PayOff('digital call',100)
callPrice = SimpleMonteCarlo2(callPayOff,1,100,0.2,0.05,10000000)
print(f'Digital Call price: {round(callPrice,4)}, for 10000000 paths')

Digital Call price: 0.5321, for 10000000 paths


In [77]:
putPayOff = PayOff('digital put',100)
putPrice = SimpleMonteCarlo2(putPayOff,1,100,0.2,0.05,10000000)
print(f'Digital Call price: {round(putPrice,4)}, for 10000000 paths')

Digital Call price: 0.4191, for 10000000 paths


Our result is quite close so we will assume it is correct.

**Exercise 2.2** Modify the pay-off class so that it can handle double digital options

The double digital options and its mechanics are described in the exercise 1.2, so here only the implementation will be showed. We will assume that Strike_ now it is a list with all the different strike prices, we can have 1 price if it is a vanilla option or 2 if it is a double digital.

In [73]:
class PayOff():
    def __init__(self,TheOptionsType_,Strike_):
        # The constructor
        self.__TheOptionsType = TheOptionsType_
        self.__Strike = Strike_
        
    def __call__(self,spot):
        # Overloading the ( ) operator
        if self.__TheOptionsType == 'call':
            return max(spot - self.__Strike[0],0)
        elif self.__TheOptionsType == 'put':
            return max(self.__Strike - spot[0],0)
        elif self.__TheOptionsType == 'double digital':
            if spot <= max(self.__Strike) and spot >= min(self.__Strike):
                return 1
            else:
                return 0           
        else:
            raise Exception('Unknown option type found.')

In [74]:
DoubleDigitalPayOff = PayOff('double digital',[120,100])
DoubleDigitalPrice = SimpleMonteCarlo2(DoubleDigitalPayOff,1,100,0.2,0.05,1000000)
print(f'Double digital option: {round(DoubleDigitalPrice,4)}, for 1000000 paths')

Double digital option: 0.3196, for 1000000 paths


Which is close to the price we were expecting.

The exercise 2.3 will not be solved since python is different than C++, and some parts not working equally, so solving that exercise it has no interest.